In [5]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
# Specify the paths to the folder containing GeoTIFF files, the shapefile, and the output folder
tif_folder = r'/efs/Uganada/SSEBop_season/Wet'
shapefile_path = r'/efs/Uganada/Boundry_AwojaOkere_Okok_1.shp'
output_folder = r'/efs/Uganada/SSEBop_season/Wet_clipped_new'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Read the shapefile using geopandas
gdf = gpd.read_file(shapefile_path)

# Iterate through each GeoTIFF file in the folder
for tif_filename in os.listdir(tif_folder):
    if tif_filename.endswith('.tif') or tif_filename.endswith('.tiff'):
        tif_path = os.path.join(tif_folder, tif_filename)

        # Open the GeoTIFF file
        with rasterio.open(tif_path) as src:
            for index, row in gdf.iterrows():
                # Get the geometry of the feature
                geom = row['geometry']

                # Mask the GeoTIFF using the shapefile geometry
                out_image, out_transform = mask(src, [geom], crop=True)
                out_image[out_image < 0] = np.nan

                # Do something with the extracted data (out_image)
                # For example, save the extracted data to a new GeoTIFF file
                output_filename = f"uganda_{tif_filename.replace('.tif', f'_boundary_{index + 1}')}.tif"
                output_path = os.path.join(output_folder, output_filename)
                with rasterio.open(output_path, 'w', driver='GTiff', height=out_image.shape[1], width=out_image.shape[2], count=1, dtype=out_image.dtype, crs=src.crs, transform=out_transform) as dst:
                    dst.write(out_image)

# Note: Make sure to install the required Python packages (`rasterio` and `geopandas`) using a package manager like `pip`.
